In [1]:
## imports 
import pandas as pd
import numpy as np
import yaml
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# comment these out if you don't have plotnine--not essential here/only used once
import matplotlib.pyplot as plt
# import plotnine
# from plotnine import *

## way to connect to mysql 
## if you need to install
## uncomment this line:
#! pip install mysql-connector-python
import mysql.connector

## function to feed path name to load
## credentials
def load_creds(path: str):
    with open(path, 'r') as stream:
        try:
            creds = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return(creds)

pd.options.display.max_rows = 999
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

/Users/vaanigupta/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


# Preliminary: define connection and read sample of data

In [18]:
## read in creds; change the path name if stored
## elsewhere
creds = load_creds("09_db_cred.yaml")

In [19]:
creds

{'practice_database': {'user': 'qss20',
  'password': 'xtPdDZHnxKMFSD5RyP',
  'port': 3306,
  'database': 'sentencing',
  'host': 'qss20-7570-db.c.dartmouth.edu'}}

In [20]:
cnx = mysql.connector.connect(user=creds['practice_database']['user'], 
                            password=creds['practice_database']['password'],
                            port=creds['practice_database']['port'],
                            database= creds['practice_database']['database'],
                            host = creds['practice_database']['host'])
cnx

# Activity 1

1. Create a new column -- `in_chicago` when pulling from the `caseinit` table that takes on the value of "YES" if INCIDENT_CITY = Chicago; "NO" otherwise (which represents incidents in Cook County suburbs outside the city limits);  and pull the table. Use `crosstabs` to confirm that this worked
2. Repeat step 1 but also filter out blank strings (`INCIDENT_CITY` == "")
3. Use `where` to row filter to initiations in Chicago and use group by to find the count of cases diverted and not diverted (`is_in_diversion`); pull the table with those counts
4. Modify the query in step 3 to find the proportion of cases in chicago diverted (hint you made need to use case when in a subquery)
5. Modify the query in step 4 to find the proportion of cases in chicago versus cases not in chicago sent to diversion 


In [21]:
# your code here 1
compare_charge_q = """
SELECT *,
       CASE
         WHEN INCIDENT_CITY = 'Chicago' THEN 'YES'
         ELSE 'NO'
       END AS in_chicago_new
FROM   caseinit 
"""

compare_charge_d = pd.read_sql_query(compare_charge_q, cnx)
compare_charge_d['in_chicago_new']

0          NO
1          NO
2          NO
3          NO
4          NO
         ... 
272289    YES
272290     NO
272291    YES
272292    YES
272293     NO
Name: in_chicago_new, Length: 272294, dtype: object

In [22]:
# your code here 2
compare_charge_q = """
SELECT *,
       CASE
         WHEN INCIDENT_CITY = 'Chicago' THEN 'YES'
         ELSE 'NO'
       END AS in_chicago_new
FROM   caseinit 
WHERE INCIDENT_CITY != ""
"""

compare_charge_d = pd.read_sql_query(compare_charge_q, cnx)
compare_charge_d[['INCIDENT_CITY', 'in_chicago_new']]

,INCIDENT_CITY,in_chicago_new
0,Oak Park,NO
1,Harvey,NO
2,Morton Grove,NO
3,Chicago,YES
4,Chicago,YES
...,...,...
251732,Chicago,YES
251733,Evergreen Park,NO
251734,Chicago,YES
251735,Chicago,YES


In [25]:
# your code here 3
t5 = """
SELECT is_in_diversion,
       Count(*) AS count_diverted
FROM   caseinit
WHERE  INCIDENT_CITY = 'Chicago'
GROUP  BY is_in_diversion
ORDER  BY count_diverted DESC
"""


t5_d = pd.read_sql_query(t5, cnx)
t5_d

,is_in_diversion,count_diverted
0,False,167171
1,True,6402


In [34]:
# your code here 4
t5 = """
SELECT avg(is_div_bool) as prop_div FROM
      (SELECT *, CASE WHEN is_in_diversion = 'True' THEN 1 ELSE 0
      END is_div_bool
      FROM caseinit) AS d
      where INCIDENT_CITY='Chicago'   
"""


t5_d = pd.read_sql_query(t5, cnx)
t5_d

,prop_div
0,0.0369


In [35]:
# your code here 5
# look at solutions for this
t5 = """
SELECT avg(is_div_bool) as prop_div,
in_chicago_new FROM
      (SELECT *, CASE WHEN is_in_diversion = 'True' THEN 1 ELSE 0
      END is_div_bool
      CASE when INCIDENT_CITY = 'Chicago' THEN 'Yes' ELSE 'No'
      END in_chicago
      FROM caseinit) AS d
      GROUP BY in_chicago_new   
"""


t5_d = pd.read_sql_query(t5, cnx)
t5_d

DatabaseError: Execution failed on sql '
SELECT avg(is_div_bool) as prop_div,
in_chicago_new FROM
      (SELECT *, CASE WHEN is_in_diversion = 'True' THEN 1 ELSE 0
      END is_div_bool
      CASE when INCIDENT_CITY = 'Chicago' THEN 'Yes' ELSE 'No'
      END in_chicago
      FROM caseinit) AS d
      GROUP BY in_chicago_new   
': 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'CASE when INCIDENT_CITY = 'Chicago' THEN 'Yes' ELSE 'No'
      END in_chicago
  ' at line 5

# Activity 2 

1. Use the following crosswalk and the `CASE` variable in the `divert` table to create a new variable `DIVERSION_PROGRAM_TEXT` that spells out the diversion programs
    - DC: Drug Court

    - DDPP: Drug Deferred Prosecution

    - DS: Drug School

    - RJCC: Restorative Justice

    - MHC: Mental Health Court

    - VC: Veteran Court

2. Build on the query from step 1 to filter to Narcotics as the `UPDATED_OFFENSE_CATEGORY` and Black or White defendants (based on race in the diversions table) (hint: you'll need to join with the caseinit table based on case_id and case_participant_id, you can do a inner join to keep only those diverted). Select the case_id, case_participant_id, case, race, and diversion_program_text columns

In [ ]:
# your code here 1

In [ ]:
# your code here 2